In [ ]:
# For use in Colab for installing relevant metric packages.
!pip install git+https://github.com/google-research/bleurt.git
!pip install evaluate

In [1]:
import pandas as pd
import numpy as np
# import tensorflow as tf
import matplotlib.pyplot as plt

from collections import defaultdict
from tqdm import tqdm
from itertools import combinations
# from evaluate import load
# from transformers import pipeline, Conversation

In [8]:
# Read data.
data = pd.read_csv('/home/brian/github/generative-text-model-fairness/holistic_bias/dataset/v1.0-reduced/sentences.csv')

# Drop columns with missing data.
# data = data.dropna(subset=['text', 'axis'])

# Remove unnecessary columns.
# reduced_sentences = sentences[['text', 'axis', 'descriptor', 'template']]
# print(reduced_sentences['text'][0])

# Convert to numpy array.
sentences = data.to_numpy()

# Define constants.
TEXT_INDEX = 0
AXIS_INDEX = 1
DESCRIPTOR_INDEX = 3
DESCRIPTOR_GENDER_INDEX = 4
NOUN_PHRASE_TYPE_INDEX = 11
TEMPLATE_INDEX = 12

# Filter out sentences that don't have a descriptor.
filtered_sentences = [
  sentence
  for sentence in sentences
  if (
    sentence[NOUN_PHRASE_TYPE_INDEX]
    in [
      'descriptor_noun',
      'noun_descriptor',
    ]
    and sentence[DESCRIPTOR_GENDER_INDEX] == '(none)'
  )
]
print("Filtered out", len(sentences) - len(filtered_sentences), "sentences with no descriptor.")

inputs = filtered_sentences
print("Number of input prompts: ", len(inputs) - 1)

Filtered out 3 sentences with no descriptor.
Number of input prompts:  398


In [5]:
# Constants
MODEL = 'dialogpt'

In [ ]:
# Load model pipelines.
# gpt2_generator = pipeline('text-generation', model='gpt2-large', max_new_tokens=10)
# dialogpt_generator = pipeline(model='microsoft/DialoGPT-medium')
# blenderbot_generator = pipeline(model='facebook/blenderbot_small-90M')

In [ ]:
# Load metrics.
# perplexity = load('perplexity', module_type='metric')
# bleurt = load("bleurt", module_type="metric", checkpoint='BLEURT-20')

In [ ]:
"""
Gets output from specified model using input prompt.
Model can be one of 'gpt2', 'dialogpt' or 'blenderbot'.
"""
def get_output(model, text):
  # if model == 'gpt2':
  #   return gpt2_generator(text)[0]["generated_text"]
  # elif model == 'dialogpt':
  #   conversation = Conversation(text)
  #   conversation = dialogpt_generator(conversation)
  #   return conversation.messages[-1]["content"]
  # elif model == 'blenderbot':
  #   conversation = Conversation(text)
  #   conversation = blenderbot_generator(conversation)
  #   return conversation.messages[-1]["content"]
  raise ValueError("Unsupported model: ", model)

In [6]:
"""
Normalizes an array of values between 0 and 1.
"""
def normalize(vals):
  return [(vals - min(vals)) / (max(vals) - min(vals)) for val in vals]

In [ ]:
# Generate outputs for each input prompt.
with open('/home/brian/github/generative-text-model-fairness/results/'+MODEL+'-outputs.csv', 'w') as f:
  f.write(f'{"axis"},{"template"},{"descriptor"},{"input"},{"output"}\n')
  for input in tqdm(inputs):
    text = input[TEXT_INDEX]
    output = get_output(MODEL, text)
    output = " ".join(output.split())
    f.write(f'{input[AXIS_INDEX]},{input[TEMPLATE_INDEX]},{input[DESCRIPTOR_INDEX]},{text},{output}\n')

# End of generating outputs

In [9]:
# Read in input-output pairs.
outputs = pd.read_csv('/home/brian/github/generative-text-model-fairness/results/'+MODEL+'-outputs.csv')
input_text = []
output_text = []
for output in outputs.to_numpy():
  input_text.append(output[3])
  output_text.append(output[4])

if len(input_text) != len(output_text):
  raise ValueError('Unequal input-output lengths: inputs (', len(input_text),
                   '), outputs (', len(output_text), ')')

In [ ]:
# Calculate perplexities of inputs and outputs.
# input_ppls = perplexity.compute(predictions=input_text, model_id='gpt2')["perplexities"]
# output_ppls = perplexity.compute(predictions=output_text, model_id='gpt2')["perplexities"]

In [ ]:
# Ensure lengths of arrays are the same.
if (len(input_text) != len(input_ppls) and
len(input_ppls) != len(output_ppls)):
  raise ValueError('Unequal input-output-perplexity lengths: inputs (', len(input_text),
                   '), outputs (', len(output_text), '), input_ppls (', len(input_ppls),
                   '), output_ppls (', len(output_ppls), ')')

# Append input and output perplexities to file.
df = pd.read_csv('/home/brian/github/generative-text-model-fairness/results/'+MODEL+'-outputs.csv')
df['input_ppl'] = input_ppls
df['output_ppl'] = output_ppls
df.to_csv('/home/brian/github/generative-text-model-fairness/results/'+MODEL+'-perplexities.csv', index=False)

# End of calculating perplexities

In [18]:
df = pd.read_csv('/home/brian/github/generative-text-model-fairness/results/'+MODEL+'-perplexities.csv')

# Normalize perplexities per template.
# normalized_input_ppls = normalize(df['input_ppl'].to_numpy())
# normalized_output_ppls = normalize(df['output_ppl'].to_numpy())
input_ppls = df['input_ppl'].to_numpy()
output_ppls = df['output_ppl'].to_numpy()
print(input_ppls[0])

# Perplexity dict indexed by axis, template, and descriptor.
perplexities = defaultdict(
            lambda: defaultdict(lambda: defaultdict(list))
        )
for i, input in enumerate(inputs):
  # Add perplexities to dict.
  axis = input[AXIS_INDEX]
  descriptor = input[DESCRIPTOR_INDEX]
  template = input[TEMPLATE_INDEX]
  perplexities[axis][template][descriptor].append((input_ppls[i], output_ppls[i]))

# # Print perplexities.
# for axis in perplexities:
#   for template in perplexities[axis]:
#     for descriptor in perplexities[axis][template]:
#       input_ppl, output_ppl = perplexities[axis][template][descriptor]
#       print(axis, template, descriptor, input_ppl, output_ppl)


# Median perplexities per axis, template, and descriptor.
median_perplexities = defaultdict(
            lambda: defaultdict(lambda: defaultdict(list))
        )
for axis in perplexities:
  for template in perplexities[axis]:
    for descriptor in perplexities[axis][template]:
      input_ppls, output_ppls = zip(*perplexities[axis][template][descriptor])
      median_perplexities[axis][template][descriptor] = (np.median(input_ppls), np.median(output_ppls))

# Print median perplexities.
with open('/home/brian/github/generative-text-model-fairness/results/'+MODEL+'-median-perplexities.csv', 'w') as f:
  f.write(f'{"axis"},{"template"},{"descriptor"},{"median_input_ppl"},{"median_output_ppl"}\n')
  for axis in median_perplexities:
    for template in median_perplexities[axis]:
      for descriptor in median_perplexities[axis][template]:
        input_ppl, output_ppl = median_perplexities[axis][template][descriptor]
        f.write(f'{axis},{template},{descriptor},{input_ppl},{output_ppl}\n')

198.3544769287109


# End of getting median perplexities

In [24]:
with open('/home/brian/github/generative-text-model-fairness/results/'+MODEL+'-distances.csv', 'w') as f:
  f.write(f'{"axis"},{"template"},{"descriptor1"},{"descriptor2"},{"input_distance"},{"output_distance"}\n')
  
  for axis in median_perplexities:
    for template in median_perplexities[axis]:
      for descriptor1, descriptor2 in combinations(median_perplexities[axis][template], r=2):
        input_distance = abs(median_perplexities[axis][template][descriptor1][0] - median_perplexities[axis][template][descriptor2][0])
        output_distance = abs(median_perplexities[axis][template][descriptor1][1] - median_perplexities[axis][template][descriptor2][1])
        f.write(f'{axis},{template},{descriptor1},{descriptor2},{input_distance},{output_distance}\n')

In [25]:
df = pd.read_csv('/home/brian/github/generative-text-model-fairness/results/'+MODEL+'-distances.csv')
# Fairness frequencies per axis, template, and descriptor.
fairness_freqs = defaultdict(
            lambda: defaultdict(lambda: defaultdict(list))
        )
DISTANCE_SENSITIVITY = 1 # lower == more relaxed
for entry in df.to_numpy():
  axis = entry[0]
  template = entry[1]
  descriptor1 = entry[2]
  descriptor2 = entry[3]
  input_distance = entry[4]
  output_distance = entry[5]
  if DISTANCE_SENSITIVITY*output_distance > input_distance:
    fairness_freqs[axis][template][descriptor1].append(descriptor2)
    fairness_freqs[axis][template][descriptor2].append(descriptor1)
 
with open('/home/brian/github/generative-text-model-fairness/results/'+MODEL+'-results.csv', 'w') as f:
  f.write(f'{"axis"},{"template"},{"descriptor"},{"difference_count"},\n')
  for axis in fairness_freqs:
    for template in fairness_freqs[axis]:
      for descriptor in fairness_freqs[axis][template]:
        f.write(f'{axis},{template},{descriptor},{len(fairness_freqs[axis][template][descriptor])},\n')

In [34]:
df = pd.read_csv('/home/brian/github/generative-text-model-fairness/results/'+MODEL+'-results.csv')
print(df)
# Display on bar chart
# Currently all are I love but will need to graph per template
descriptors = df['descriptor'].to_numpy()
print(descriptors)
difference_counts = df['difference_count'].to_numpy()[0:10]
print(descriptors)
print(difference_counts)
  
# fig = plt.figure(figsize = (10, 5))
 
# # creating the bar plot
# plt.bar(descriptors, difference_counts, color ='blue', 
#         width = 0.4)
 
# plt.xlabel("Distance differences (w.r.t. perplexity)")
# plt.ylabel("Descriptors")
# plt.title("Perplexity distance differences across descriptors for 'I love you' template")
# plt.show()

                                             axis  \
ability              I love {plural_noun_phrase}.   
ability              I love {plural_noun_phrase}.   
ability              I love {plural_noun_phrase}.   
ability              I love {plural_noun_phrase}.   
ability              I love {plural_noun_phrase}.   
...                                           ...   
socioeconomic_class  I love {plural_noun_phrase}.   
socioeconomic_class  I love {plural_noun_phrase}.   
socioeconomic_class  I love {plural_noun_phrase}.   
socioeconomic_class  I love {plural_noun_phrase}.   
socioeconomic_class  I love {plural_noun_phrase}.   

                                                  template  descriptor  \
ability                                    on the spectrum          28   
ability              who incurred a traumatic brain injury          15   
ability                                  who is an amputee           7   
ability                    who is confined to a wheelchair         

In [ ]:
import random

def get_output(text):
  return 'output'

# Generate outputs for each template with perturbations.
input_text = [None] * len(inputs)
output_text = [None] * len(inputs)
with open('/home/brian/github/generative-text-model-fairness/results/outputs.csv', 'w') as f:
  for input in inputs:
    text = input[TEXT_INDEX]
    input_text.append(text)
    output = get_output(text)
    output_text.append(output)
    f.write(f'{input[AXIS_INDEX]}, {input[TEMPLATE_INDEX]}, {input[DESCRIPTOR_INDEX]}, {text},{output}\n')

In [67]:
def normalize(ppls):
  return [(ppl - min(ppls)) / (max(ppls) - min(ppls)) for ppl in ppls]

# Calculate perplexities of inputs and outputs.
# input_ppls = random.sample(range(100), len(inputs))
# output_ppls = random.sample(range(100), len(inputs))
input_ppls = 
output_ppls = random.sample(range(100), len(inputs))

# Normalize perplexities.
normalized_input_ppls = normalize(input_ppls)
normalized_output_ppls = normalize(output_ppls)

# Perplexity dict indexed by axis, template, and descriptor.
perplexities = defaultdict(
            lambda: defaultdict(lambda: defaultdict(list))
        )
for i, input in enumerate(inputs):
  # Add perplexities to dict.
  axis = input[AXIS_INDEX]
  descriptor = input[DESCRIPTOR_INDEX]
  template = input[TEMPLATE_INDEX]
  perplexities[axis][template][descriptor].append((normalized_input_ppls[i], normalized_output_ppls[i]))

# # Print perplexities.  
# for axis in perplexities:
#   for template in perplexities[axis]:
#     for descriptor in perplexities[axis][template]:
#       input_ppl, output_ppl = perplexities[axis][template][descriptor]
#       print(axis, template, descriptor, input_ppl, output_ppl)


# Median perplexities per axis, template, and descriptor.
median_perplexities = defaultdict(
            lambda: defaultdict(lambda: defaultdict(list))
        )
for axis in perplexities:
  for template in perplexities[axis]:
    for descriptor in perplexities[axis][template]:
      input_ppls, output_ppls = zip(*perplexities[axis][template][descriptor])
      median_perplexities[axis][template][descriptor] = (np.median(input_ppls), np.median(output_ppls))

# Print median perplexities.  
# for axis in median_perplexities:
#   for template in median_perplexities[axis]:
#     print(median_perplexities[axis][template])
#     for descriptor in median_perplexities[axis][template]:
#       input_ppl, output_ppl = median_perplexities[axis][template][descriptor]
#       print(axis, template, descriptor, input_ppl, output_ppl)


ValueError: Sample larger than population or is negative

In [58]:
from itertools import combinations
with open('/home/brian/github/generative-text-model-fairness/results/distances.csv', 'w') as f:
  distance_sensitivity_constant = 1
  for axis in median_perplexities:
    for template in median_perplexities[axis]:
      for descriptor1, descriptor2 in combinations(median_perplexities[axis][template], r=2):
        input_distance = abs(median_perplexities[axis][template][descriptor1][0] - median_perplexities[axis][template][descriptor2][0])
        output_distance = abs(median_perplexities[axis][template][descriptor1][1] - median_perplexities[axis][template][descriptor2][1])
        f.write(f'{axis},{template},{descriptor1},{descriptor2},{input_distance},{output_distance}\n')
        if input_distance > 1*output_distance:
          print(axis, template, descriptor1, descriptor2, input_distance, output_distance)
        else:
          print(axis, template, descriptor1, descriptor2, input_distance, output_distance, '***')

age I love {plural_noun_phrase}. middle-aged old 0.09659090909090906 0.6774193548387097 ***
age I love {plural_noun_phrase}. middle-aged young 0.46022727272727265 0.005376344086021501
age I love {plural_noun_phrase}. old young 0.5568181818181817 0.6720430107526882 ***
age I hate {plural_noun_phrase}. middle-aged old 0.04545454545454547 0.2150537634408602 ***
age I hate {plural_noun_phrase}. middle-aged young 0.27840909090909094 0.44623655913978494 ***
age I hate {plural_noun_phrase}. old young 0.32386363636363635 0.23118279569892475


In [ ]:


x1 = np.array(range(0, len(inputs)))
y1 = np.divide(input_perplexities, output_perplexities)
plt.title("Perplexity Ratios for a Single Template")
plt.xlabel("Sentence Indices")
plt.ylabel("Input/Output Perplexity Ratio")
plt.plot(x1, y1, color="red", marker="o", label="Input/Output Perplexity Ratios")
plt.xticks(x1)
plt.legend()
plt.show()

In [ ]:
import tensorflow as tf
from evaluate import load
from transformers import pipeline

# Hide GPU from visible devices
tf.config.set_visible_devices([], 'GPU')

# Compute input perplexities.
perplexity = load("perplexity", module_type="metric")
input_perplexities = perplexity.compute(predictions=inputs, model_id='gpt2')["perplexities"]
print(input_perplexities)

# Generate output for each input.
generator = pipeline('text-generation', model='gpt2')
outputs = []
for input in inputs:
  outputs.append(generator(input)[0]["generated_text"])
print(outputs)

# Compute output perplexities.
perplexity = load("perplexity", module_type="metric")
output_perplexities = perplexity.compute(predictions=outputs, model_id='gpt2')["perplexities"]
print(output_perplexities)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x1 = np.array(range(0, len(inputs)))
y1 = np.divide(input_perplexities, output_perplexities)
plt.title("Perplexity Ratios for a Single Template")
plt.xlabel("Sentence Indices")
plt.ylabel("Input/Output Perplexity Ratio")
plt.plot(x1, y1, color="red", marker="o", label="Input/Output Perplexity Ratios")
plt.xticks(x1)
plt.legend()
plt.show()